In [28]:
from main import load_scm
import pickle
import numpy as np
import os
import random
from sklearn.metrics import roc_curve
from transformation_utils import apply_point_to_set_transformation, calculate_dij_and_its_eigenvalues_and_eigenvectors, calculate_Dij

In [2]:
SCM_GPDS_PATH = 'SCM_NEW_FEATURES_12X12_GPDS'

In [40]:
gpds_dataset_sample = load_scm(SCM_GPDS_PATH, num_of_writers_to_load=100)

In [11]:
with open(os.path.join('TRAINING_RESULTS_CEDAR_PROTOCOL_2_12_12', 'forgery_mixed_individuals_n12_m1_p12_writers55_epoch200_max_iterations10000_batch400_27022024_184230.pkl'), 'rb') as pickle_file:
    loaded_model = pickle.load(pickle_file)

In [25]:
loaded_model['fold_results'][2].keys()

dict_keys(['W', 'M', 'A', 'aucs', 'val_eers', 'train_loss', 'val_loss', 'test_eer', 'test_users'])

In [12]:
def get_batch(datasource, num_elements, individuals_curr_fold, same_or_mixed, origin_list_1, origin_list_2):

        list_of_already_selected = []
        list_of_pairs = []

        first_pair_element_max_index = 24
        second_pair_element_max_index = 24

        if same_or_mixed == 'same':
            while len(list_of_pairs) < num_elements:
                random_individual = random.choice(individuals_curr_fold)
                first_pair_element = random.randint(0, first_pair_element_max_index - 1)
                second_pair_element = random.randint(0, second_pair_element_max_index - 1)

                while first_pair_element_max_index == second_pair_element_max_index and first_pair_element == second_pair_element:
                    second_pair_element = random.randint(0, second_pair_element_max_index - 1)

                identifier = (random_individual, first_pair_element, second_pair_element)

                if identifier not in list_of_already_selected:
                    list_of_already_selected.append(identifier)
                    pair = (datasource[random_individual][origin_list_1][first_pair_element],
                            datasource[random_individual][origin_list_2][second_pair_element])
                    list_of_pairs.append(pair)

        elif same_or_mixed == 'mixed':
            while len(list_of_pairs) < num_elements:
                random_individual_1 = random.choice(individuals_curr_fold)
                random_individual_2 = random.choice(individuals_curr_fold)
                
                while random_individual_1 == random_individual_2:
                    random_individual_2 = random.choice(individuals_curr_fold)

                first_pair_element = random.randint(0, first_pair_element_max_index - 1)
                second_pair_element = random.randint(0, second_pair_element_max_index - 1)

                identifier = (random_individual_1, random_individual_2, first_pair_element, second_pair_element)

                if identifier not in list_of_already_selected:
                    list_of_already_selected.append(identifier)
                    pair = (datasource[random_individual_1][origin_list_1][first_pair_element],
                            datasource[random_individual_2][origin_list_2][second_pair_element])
                    list_of_pairs.append(pair)
                    
        return np.array(list_of_pairs)

In [41]:
s_pairs_test = get_batch(gpds_dataset_sample, 1000, list(gpds_dataset_sample.keys()), 'same', 'genuine_scm', 'genuine_scm')

In [42]:
d_pairs_test = get_batch(gpds_dataset_sample, 1000, list(gpds_dataset_sample.keys()), 'mixed', 'genuine_scm', 'genuine_scm')

In [43]:
d_pairs_skilled_test = get_batch(gpds_dataset_sample, 1000, list(gpds_dataset_sample.keys()), 'same', 'genuine_scm', 'forged_scm')

In [30]:
def run_test(loaded_model, s_pairs_test, d_pairs_test, selected_fold):
    m = loaded_model['m']
    n = loaded_model['n']
    p = loaded_model['p']
    zeta_s = loaded_model['zeta_s']
    zeta_d = loaded_model['zeta_d']
    
    W = loaded_model['fold_results'][selected_fold]['W']
    M = loaded_model['fold_results'][selected_fold]['M']
    A = loaded_model['fold_results'][selected_fold]['A']

    s_pairs_of_sets_test = apply_point_to_set_transformation(s_pairs_test, W, m, p)
    d_pairs_of_sets_test = apply_point_to_set_transformation(d_pairs_test, W, m, p)

    S_dij_test, _, _ = calculate_dij_and_its_eigenvalues_and_eigenvectors(s_pairs_of_sets_test, A)
    D_dij_test, _, _ = calculate_dij_and_its_eigenvalues_and_eigenvectors(d_pairs_of_sets_test, A)

    S_Dij_test = calculate_Dij(S_dij_test, M)
    D_Dij_test = calculate_Dij(D_dij_test, M)
    total_test = np.concatenate((S_Dij_test, D_Dij_test))
    labels_test = np.concatenate((np.ones_like(S_Dij_test), np.zeros_like(D_Dij_test)))

    probabilities = np.zeros_like(total_test, dtype=float)
    probabilities[total_test < zeta_s] = 1.0
    probabilities[total_test > zeta_d] = 0.0
    in_between_mask = (total_test >= zeta_s) & (total_test <= zeta_d)
    probabilities[in_between_mask] = 1 - ((total_test[in_between_mask] - zeta_s) / (zeta_d - zeta_s))

    fpr, tpr, _ = roc_curve(labels_test, probabilities)
    fnr = 1 - tpr
    EER = fpr[np.nanargmin(np.absolute((fnr - fpr)))]

    return EER

In [44]:
EER_test_mixed = run_test(loaded_model, s_pairs_test, d_pairs_test, 1)
EER_test_mixed

0.312

In [45]:
EER_test_same = run_test(loaded_model, s_pairs_test, d_pairs_skilled_test, 1)
EER_test_same

0.353